## Estadistica Aplicada 

Sesion 11 - 19 septiembre 

Ana Sofia Saucedo Garcia
Grupo 42

In [5]:
# #Librerias basicas

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline 

In [6]:
#Libreria especializada

from lifelines import CoxPHFitter, ExponentialFitter, WeibullFitter, LogNormalFitter
from lifelines.datasets import load_rossi


In [9]:
# datos 

rossi = load_rossi()

In [10]:
rossi.head()

,week,arrest,fin,age,race,wexp,mar,paro,prio
0,20,1,0,27,1,0,0,1,3
1,17,1,0,18,1,0,0,1,8
2,25,1,0,19,0,1,0,1,13
3,52,0,1,23,1,1,1,1,1
4,52,0,0,19,0,1,0,1,3


In [12]:
# estimacion del modelo 

cph1 = CoxPHFitter().fit(rossi,'week','arrest')
cph1.print_summary()

<lifelines.CoxPHFitter: fitted with 432 total observations, 318 right-censored observations>
             duration col = 'week'
                event col = 'arrest'
      baseline estimation = breslow
   number of observations = 432
number of events observed = 114
   partial log-likelihood = -658.75
         time fit was run = 2023-09-19 22:27:42 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
fin        -0.38       0.68       0.19            -0.75            -0.00                 0.47                 1.00
age        -0.06       0.94       0.02            -0.10            -0.01                 0.90                 0.99
race        0.31       1.37       0.31            -0.29             0.92                 0.75                 2.50
wexp       -0.15       0.86       0.21            -0.57             0.27                 0.57                 1.30
mar        -0.43       0.65       0.38            -1.18             0.31                 0.31                 1.37
paro       -0.08       0.92       0.20            -0.47             0.30                 0.63                 1.35
prio        0.09       1.10       0.03             0.04             0.15                 1.04                 1.16

            cmp to     z      p   -log2(p)
covariate                                 
fin           0.00 -1.98   0.05       4.40
age           0.00 -2.61   0.01       6.79
race          0.00  1.02   0.31       1.70
wexp          0.00 -0.71   0.48       1.06
mar           0.00 -1.14   0.26       1.97
paro          0.00 -0.43   0.66       0.59
prio          0.00  3.19 <0.005       9.48
---
Concordance = 0.64
Partial AIC = 1331.50
log-likelihood ratio test = 33.27 on 7 df
-log2(p) of ll-ratio test = 15.37

El modelo de Cox esta construido sobre el supuesto de riesgo proporcional 

Tenemos que verificar que el supuesto se cumpla 

In [14]:
cph1.check_assumptions(rossi,p_value_threshold=0.05)


# H_0: Se cumple riesgo proporcional constante 
# H_a: No Se cumple riesgo proporcional constante 

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 432 total observations, 318 right-censored observations>
         test_name = proportional_hazard_test

---
           test_statistic      p  -log2(p)
age  km             11.03 <0.005     10.12
     rank           11.45 <0.005     10.45
fin  km              0.02   0.89      0.17
     rank            0.02   0.90      0.15
mar  km              0.60   0.44      1.19
     rank            0.71   0.40      1.32
paro km              0.12   0.73      0.45
     rank            0.13   0.71      0.49
prio km              0.02   0.88      0.18
     rank            0.02   0.89      0.17
race km              1.44   0.23      2.12
     rank            1.43   0.23      2.11
wexp km              7.48   0.01      7.32
     rank            7.31   0.01      7.19



1. Variable 'age' failed the non-proportional test: p-value is 0.0007.

   Advice 1: the functional form of the variable 'age' might be incorrect. That is, there may be
non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'age' using pd.cut, and then specify it in `strata=['age',
...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'wexp' failed the non-proportional test: p-value is 0.0063.

   Advice: with so few unique values (only 2), you can include `strata=['wexp', ...]` in the call in
`.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/lat

[]

Si alguna variable no cumple el supuesto , entonces no se puede utilizar el modelo Cox 

Hay dos opciones : 1 - Eliminar las variables que violan el supuesto o 2- Utilizar otro modelo (Modelo de Riesgo Acelerado AFT ).
Para usar el modelo de riesgo acelerado primero tenemos que verificar si es Weibull , exponencial o Lognormal. Y luego estimar el modelo AFT 

In [17]:
mexpo = ExponentialFitter().fit(rossi['week'],rossi['arrest'])
mweibull = WeibullFitter().fit(rossi['week'],rossi['arrest'])
mlogn = LogNormalFitter().fit(rossi['week'],rossi['arrest'])

El criterio para la elección sugiere aplicar el que tiene menor valor de AIC

In [18]:
mexpo.AIC_, mweibull.AIC_ , mlogn.AIC_

(1405.9540519867492, 1397.248793895584, 1399.8208496762338)

In [20]:
# Dado el valor del AIC significa que debe ser un modelo AFT weibull

from lifelines import WeibullAFTFitter 
weibullAFT = WeibullAFTFitter().fit(rossi,'week','arrest')
weibullAFT.print_summary()


<lifelines.WeibullAFTFitter: fitted with 432 total observations, 318 right-censored observations>
             duration col = 'week'
                event col = 'arrest'
   number of observations = 432
number of events observed = 114
           log-likelihood = -679.92
         time fit was run = 2023-09-19 22:42:00 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param   covariate                                                                                                         
lambda_ age         0.04       1.04       0.02             0.01             0.07                 1.01                 1.07
        fin         0.27       1.31       0.14             0.00             0.54                 1.00                 1.72
        mar         0.31       1.37       0.27            -0.22             0.85                 0.80                 2.33
        paro        0.06       1.06       0.14            -0.21             0.33                 0.81                 1.39
        prio       -0.07       0.94       0.02            -0.11            -0.02                 0.90                 0.98
        race       -0.22       0.80       0.22            -0.66             0.21                 0.52                 1.23
        wexp        0.11       1.11       0.15            -0.19             0.40                 0.83                 1.50
        Intercept   3.99      54.06       0.42             3.17             4.81                23.78               122.92
rho_    Intercept   0.34       1.40       0.09             0.16             0.51                 1.18                 1.67

                    cmp to     z      p   -log2(p)
param   covariate                                 
lambda_ age           0.00  2.54   0.01       6.51
        fin           0.00  1.97   0.05       4.37
        mar           0.00  1.14   0.25       1.97
        paro          0.00  0.42   0.67       0.57
        prio          0.00 -3.14 <0.005       9.22
        race          0.00 -1.02   0.31       1.70
        wexp          0.00  0.70   0.48       1.05
        Intercept     0.00  9.52 <0.005      68.98
rho_    Intercept     0.00  3.81 <0.005      12.81
---
Concordance = 0.64
AIC = 1377.83
log-likelihood ratio test = 33.42 on 7 df
-log2(p) of ll-ratio test = 15.46

Se rechaza H_0 : La variable edad tiene efecto (estadisticamente significativa) ,la edad aumenta la mediana del tiempo de supervivencia 4% por cada año de edad 

